# UDX Worker Interface

This notebook serves as the interface for the UDX worker. It provides the necessary setup and function registrations. It also includes a basic local testing of operational tasks running a UDX worker based Docker container.

In [ ]:
const fs = require('fs');
const path = require('path');
const { Docker } = require('dockerode');

// Initialize the Docker client
const docker = new Docker();

// Your JavaScript code
const jsCode = `
/**
 * 
 * Mock service that logs a message every 5 seconds.
 * 
 */

function sleep(seconds) {
    const milliseconds = seconds * 1000;
    const start = Date.now();
    while (true) {
      const current = Date.now();
      if (current - start >= milliseconds) {
        break;
      }
    }
}

while (true) {
    console.log(\`[\${new Date().toLocaleTimeString()}] Service is up and running!\`);
    sleep(5);
}
`;

// Write the JavaScript code to a file
const scriptPath = path.join(__dirname, 'script.js');
fs.writeFileSync(scriptPath, jsCode);

// Function to run the Docker container
async function runContainer() {
    try {
        // Pull the Docker image
        // Replace 'your-docker-image' with the name of your Docker image
        const image = await docker.pull('your-docker-image');
        console.log('Image pulled:', image);

        // Create and start the container
        const container = await docker.createContainer({
            Image: 'your-docker-image',
            Cmd: ['node', '/usr/src/app/script.js'],
            Volumes: {
                '/usr/src/app': {}
            },
            HostConfig: {
                Binds: [`${__dirname}:/usr/src/app`]
            }
        });

        await container.start();
        console.log('Container started:', container.id);

        // Fetch the logs of the container
        const logs = await container.logs({
            follow: true,
            stdout: true,
            stderr: true
        });

        logs.on('data', (data) => {
            console.log(data.toString());
        });

    } catch (error) {
        console.error('Error:', error);
    }
}

// Run the container
runContainer();
